In [1]:
import yfinance as yf
import pandas as pd
import sqlite3
import os

# --- 1. ÁREA DE CONFIGURAÇÃO (FÁCIL DE EDITAR) ---

TICKERS = [
    'PETR4.SA', 'VALE3.SA', 'ITUB4.SA', 'BBAS3.SA', 'BBDC4.SA', 
    'ABEV3.SA', 'B3SA3.SA', 'ELET3.SA', 'WEGE3.SA', 'RENT3.SA',
    'BPAC11.SA', 'RADL3.SA', 'ITSA4.SA', 'PRIO3.SA', 'MGLU3.SA',
    'UGPA3.SA', 'GGBR4.SA', 'ASAI3.SA', 'SUZB3.SA', 'VBBR3.SA',
    'CPLE6.SA', 'LREN3.SA', 'KLBN11.SA', 'RAIL3.SA', 'BBSE3.SA',
    'EQTL3.SA', 'CMIG4.SA', 'RDOR3.SA', 'EMBR3.SA', 'SBSP3.SA',
    'VAMO3.SA', 'BEEF3.SA', 'CSAN3.SA', 'COGN3.SA', 'TOTS3.SA',
    'TIMS3.SA', 'FLRY3.SA', 'PETR3.SA', 'USIM5.SA', 'BRAV3.SA',
    'NEOE3.SA', 'CSNA3.SA', 'AURE3.SA', 'MRVE3.SA', 'AZUL4.SA',
    'GMAT3.SA', 'MBRF3.SA', 'PCAR3.SA', 'VIVA3.SA', 'GOAU4.SA',
    'YDUQ3.SA', 'CVCB3.SA', 'LWSA3.SA', 'MOTV3.SA',
    'ANIM3.SA', 'NATU3.SA', 'BBDC3.SA', 'ENEV3.SA', 'ONCO3.SA',
    'CPLE3.SA', 'HBSA3.SA', 'AMBP3.SA', 'RAIZ4.SA', 'AZZA3.SA', 
    'LIGT3.SA', 'SULA11.SA', 'HYPE3.SA', 'JHSF3.SA', 'SMLS3.SA',
    'CSED3.SA', 'TUPY3.SA', 'BRKM5.SA', 'CASH3.SA'
]
DATABASE_FILE = 'dados_acoes.db'
PERIODO_DADOS = 'max'

# --- FIM DA CONFIGURAÇÃO ---


# 2. Dicionário dos intervalos
intervalos = {
    'candles_diarios': '1d',
    'candles_semanais': '1wk',
    'candles_mensais': '1mo'
}

# 3. Conexão com o banco de dados
print(f"Iniciando conexão com o banco de dados '{DATABASE_FILE}'...")
conn = sqlite3.connect(DATABASE_FILE)

try:
    # 4. Loop principal para baixar os dados
    for tabela_sql, intervalo_yf in intervalos.items():
        print(f"\nBuscando dados para: {tabela_sql} (Intervalo: {intervalo_yf})")
        
        lista_dfs_por_ticker = []
        
        for ticker in TICKERS:
            print(f"  Baixando {ticker}...")
            
            # Baixa os dados usando yfinance
            data = yf.download(ticker, period=PERIODO_DADOS, interval=intervalo_yf)
            
            # --- INÍCIO DA CORREÇÃO ---
            # Resolve o problema de MultiIndex (se houver)
            if isinstance(data.columns, pd.MultiIndex):
                data.columns = data.columns.get_level_values(0)
            # --- FIM DA CORREÇÃO ---
            
            if data.empty:
                print(f"  Sem dados para {ticker} no intervalo {intervalo_yf}.")
                continue
                
            # Adiciona o nome do ticker como uma coluna
            data['Ticker'] = ticker
            
            # Transforma o índice (Date) em uma coluna
            data.reset_index(inplace=True)
            
            # Renomeia a coluna de data para ser mais clara
            if 'Date' in data.columns:
                data.rename(columns={'Date': 'Datetime'}, inplace=True)
            
            lista_dfs_por_ticker.append(data)
            
        # 5. Combina todos os dataframes
        if not lista_dfs_por_ticker:
            print(f"Nenhum dado baixado para o intervalo {intervalo_yf}.")
            continue
            
        df_completo = pd.concat(lista_dfs_por_ticker, ignore_index=True)
        
        # 6. Limpa e padroniza os nomes das colunas para SQL
        # Esta linha agora vai funcionar!
        df_completo.columns = df_completo.columns.str.lower().str.replace(' ', '_')
        
        # 7. Salva o dataframe combinado na tabela SQL
        print(f"  Salvando dados na tabela SQL: '{tabela_sql}'...")
        df_completo.to_sql(
            tabela_sql, 
            conn, 
            if_exists='replace',
            index=False
        )
        
        print(f"  Tabela '{tabela_sql}' salva com sucesso!")

    print(f"\nProcesso concluído! Todos os dados foram salvos em '{DATABASE_FILE}'.")

except Exception as e:
    print(f"\nOcorreu um erro: {e}")

finally:
    # 8. Fecha a conexão com o banco de dados
    conn.close()
    print("Conexão com o banco de dados fechada.")

Iniciando conexão com o banco de dados 'dados_acoes.db'...

Buscando dados para: candles_diarios (Intervalo: 1d)
  Baixando PETR4.SA...


C:\Users\Gustavo\AppData\Local\Temp\ipykernel_20712\2292063328.py:53: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period=PERIODO_DADOS, interval=intervalo_yf)
[*********************100%***********************]  1 of 1 completed
C:\Users\Gustavo\AppData\Local\Temp\ipykernel_20712\2292063328.py:53: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period=PERIODO_DADOS, interval=intervalo_yf)


  Baixando VALE3.SA...


[*********************100%***********************]  1 of 1 completed
C:\Users\Gustavo\AppData\Local\Temp\ipykernel_20712\2292063328.py:53: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period=PERIODO_DADOS, interval=intervalo_yf)


  Baixando ITUB4.SA...


...

  Baixando BRKM5.SA...


[*********************100%***********************]  1 of 1 completed
C:\Users\Gustavo\AppData\Local\Temp\ipykernel_20712\2292063328.py:53: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period=PERIODO_DADOS, interval=intervalo_yf)


  Baixando CASH3.SA...


[*********************100%***********************]  1 of 1 completed
C:\Users\Gustavo\AppData\Local\Temp\ipykernel_20712\2292063328.py:53: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period=PERIODO_DADOS, interval=intervalo_yf)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['IGTA3.SA']: YFTzMissingError('possibly delisted; no timezone found')


  Baixando IGTA3.SA...
  Sem dados para IGTA3.SA no intervalo 1mo.
  Salvando dados na tabela SQL: 'candles_mensais'...
  Tabela 'candles_mensais' salva com sucesso!

Processo concluído! Todos os dados foram salvos em 'dados_acoes.db'.
Conexão com o banco de dados fechada.
